This is the notebook to run the seq2seq network. It starts with a preprocessed file with p2_calib being the value to predict, and int_deliv_inv_ub and calib_time being the best values to use. It's important to note that at this point, some of the lumi measurements are taken a bit far from the calibration times. this can be seen by the difference between the lumi-section time and the calib_time values. The best data probably just consistents of the points where these are close.

All data is in its original units.

Also, I'm running this in the Python 3.7.1 Kernel

In [15]:
#import the stuff
import pandas as pd #dataframes etc
import matplotlib.pyplot as plt #plotting
import numpy as np
from sklearn import preprocessing
from common.utils import TimeSeriesTensor, create_evaluation_df, mape, scale_shrinker
#now we bring in the keras tensorflow model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredLogarithmicError

from math import pow, floor
import time

In [28]:
#load the data
test = pd.read_csv('../data/test_diff.csv', index_col=0)
valid = pd.read_csv('../data/valid_diff.csv', index_col=0)
train = pd.read_csv('../data/train_diff.csv', index_col=0)
#set index to datetime periods
#test.index = pd.to_datetime(test.index).to_period('H')
#valid.index = pd.to_datetime(valid.index).to_period('H')
#train.index = pd.to_datetime(train.index).to_period('H')
#set index to datetime
test.index = pd.to_datetime(test.index)
valid.index = pd.to_datetime(valid.index)

train = train.append(valid)
train.index = pd.to_datetime(train.index)
train.head()

,p2,int_deliv_inv_ub
2018-05-12 09:01:31,0.000267,1.912590e+07
2018-05-12 10:01:31,0.000199,1.786448e+07
2018-05-12 11:01:31,0.000203,1.664305e+07
2018-05-12 12:01:31,0.000395,3.387666e+07
2018-05-12 13:01:31,0.000471,3.200991e+07


In [29]:
#now we will scale the data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
y_scaler = StandardScaler() #we'll use the y-scaler later
y_scaler.fit(train[['p2']])
train[['p2', 'int_deliv_inv_ub']] = scaler.fit_transform(train)
train.describe()


,p2,int_deliv_inv_ub
count,3.734000e+03,3.734000e+03
mean,-2.942061e-17,-4.524441e-16
std,1.000134e+00,1.000134e+00
min,-4.202259e+00,-8.722823e-01
25%,-1.913427e-01,-8.722823e-01
50%,1.828191e-01,-4.994302e-01
75%,5.939159e-01,8.539115e-01
max,3.162806e+00,2.434024e+00


In [30]:
test[['p2', 'int_deliv_inv_ub']] = scaler.transform(test)
test.describe()

,p2,int_deliv_inv_ub
count,1166.000000,1166.000000
mean,0.030238,-0.604280
std,0.607155,0.677192
min,-3.891640,-0.872282
25%,-0.089422,-0.872282
50%,0.060977,-0.872282
75%,0.257269,-0.872281
max,2.381403,2.171513


In [ ]:
#build the data
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [6]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

#Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


[[ 0.59420064  0.25958877]
 [ 0.44594011  0.18332401]
 [ 0.45604353  0.10947734]
 [ 0.87168012  1.15141108]
 [ 1.03762608  1.0385484 ]
 [ 0.68422353  0.92382076]
 [ 0.38075093  0.62462487]
 [ 0.89124915  0.35741122]
 [ 0.32194582  0.3213169 ]
 [-1.18816483  0.66245371]]
[-1.67139609]


(2645, 10, 2)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")
